<a href="https://colab.research.google.com/github/carlibeisel/pod_pou_lulcc/blob/main/02_extract_gridmet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Modified by Carli Beisel

Adapted from code written by Bridget Bittmann (2023, Github: bridgetmarie24)

Date originally created: March 28, 2022

Date modified: April 16, 2024

Purpose: This script extracts DayMET data from Google Earth Engine using the Earth Engine API in Python. This script also copied over a SSEBop ET data dataset and calculated zonal stats based on polygons.

In [ ]:
# Installs geemap package
import subprocess
!pip install geemap #added

try:
    import geemap
except ImportError:
    print('geemap package not installed. Installing ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])

# Checks whether this notebook is running on Google Colab
try:
    import google.colab
    import geemap.eefolium as emap
except:
    import geemap as emap

# Authenticates and initializes Earth Engine
import ee

try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize(project = 'extract-gridmet') #input and create project on google earth engiine

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 7.9 MB/s eta 0:00:00


In [ ]:
!pip install geopandas
import geopandas as gpd #import independent shapefile
import json #for metadata of shapefile
import os #for file paths
import numpy as np #for stats and arrays
import pandas as pd #for dataframes

In [ ]:
#Connect to Google Drive if you want to export images
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
## ------------------------------------------------------- ##
## 1. Import shapefile and start/end dates to clip dataset ##
## ------------------------------------------------------- ##
!pip install pycrs

start_end = pd.read_csv('/content/drive/MyDrive/Data/pod_pou_lulcc/data_output/diversion_timeseries_out/se_dates.csv')
shp_file = '/content/drive/MyDrive/Data/pod_pou_lulcc/data_input/POUs/POUs_EDIT_060622_Merge.shp'
subset = emap.shp_to_ee(shp_file) # converts shapefile to feature in GEE
bpbc = emap.shp_to_ee('/content/drive/MyDrive/Data/pod_pou_lulcc/data_input/BPBC_Divisions/BPBC_Divisions-IDTM.shp')

map=emap.Map(center=(43.6150, -116.2023),zoom=8)
map.addLayer(ee.Image().paint(bpbc, 0, 2), {}, 'POU')
map.addLayerControl()
map

Map(center=[43.615, -116.2023], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchD…

The following code uses a GEE script created by Bridget Bittmann to find ET for each diversion region using SSEBop. Keep the same file path to ensure that you are using the correct GEE code.

In [ ]:
## ---------------------------------------------------- ##
## 2. IMPORT PRECIP, TEMP, and ET FOR IRRIGATION SEASON ##
## ---------------------------------------------------- ##

years = np.arange(1987,2021)

# Empty lists to store images
ir_tmp = []
ir_pr = []
mean_max = []
et_irrig = []
mar_tmp = []
mar_prcp = []
mar_et = []

# Import image collection, subset to shapefile, take a statistics for a period of time,
# and append the image to the designated list
for i in range(len(years)):
  mar = ee.ImageCollection("NASA/ORNL/DAYMET_V4").filterDate((str(years[i])+'-03-01'), (str(years[i])+'-03-31'))
  m_tmp  = mar.select('tmax').map(lambda image: image.clip(subset)).mean().set({'system:index': (str(years[i])+'-03')})
  m_prcp  = mar.select('prcp').map(lambda image: image.clip(subset)).mean().set({'system:index': (str(years[i])+'-03')})
  et_mar = ee.ImageCollection('projects/earthengine-legacy/assets/users/bridgetbittmann/ssebop/boise').filterDate((str(years[i])+'-03-01'), (str(years[i])+'-3-31'))
  et_m = et_mar.map(lambda image: image.clip(subset)).sum().multiply(0.00001).set({'system:index': str(years[i])})
  daymet = ee.ImageCollection("NASA/ORNL/DAYMET_V4").filterDate(start_end['StartDate'][i], start_end['EndDate'][i]) #get image collection for irrigation season
  daymet_hot = ee.ImageCollection("NASA/ORNL/DAYMET_V4").filterDate((str(years[i])+'-06-01'), (str(years[i])+'-8-31')) #get image collection for June-Aug
  mxtmp = daymet_hot.select('tmax').map(lambda image: image.clip(subset)).mean().set({'system:index':str(years[i])}) #select temp to analyze hot months and take mean
  tmp = daymet.select('tmax').map(lambda image: image.clip(subset)).mean().set({'system:index':str(start_end['StartDate'][i])}) #select max temp to analyze and take mean
  pr = daymet.select('prcp').map(lambda image: image.clip(subset)).sum().set({'system:index': str(start_end['StartDate'][i])}) #select precip to analyze and sum
  ir_tmp.append(tmp)
  ir_pr.append(pr)
  mean_max.append(mxtmp)
  mar_tmp.append(m_tmp)
  mar_prcp.append(m_prcp)
  mar_et.append(et_m)
  start = start_end[start_end['Year'] == years[i]]
  print(int(start['StartDayofYear'].head(1)))
  if int(start['StartDayofYear'].head(1)) < 91:
    et_data = ee.ImageCollection('projects/earthengine-legacy/assets/users/bridgetbittmann/ssebop/boise').filterDate((str(years[i])+'-03-01'), str(years[i])+'-10-31')
    et = et_data.map(lambda image: image.clip(subset)).sum().multiply(0.00001).set({'system:index': str(years[i])}) # sum et and convert to meters
  else:
    et_data = ee.ImageCollection('projects/earthengine-legacy/assets/users/bridgetbittmann/ssebop/boise').filterDate((str(years[i])+'-04-01'), str(years[i])+'-10-31')
    et = et_data.map(lambda image: image.clip(subset)).sum().multiply(0.00001).set({'system:index': str(years[i])}) # sum et and convert to meters
  et_irrig.append(et)


# Convert lists of images to image collection for zonal stats command
et_irrig = ee.ImageCollection(et_irrig)
ir_tmp = ee.ImageCollection(ir_tmp)
ir_pr = ee.ImageCollection(ir_pr)
means_max_temp = ee.ImageCollection(mean_max)

91
64
91
78
91
92
91
91
91
92
62
82
91
73
91
91
91
92
91
88
91
92
64
81
91
73
80
83
61
81
80
77
63
70


In [ ]:
# Convert lists of images to image collection for zonal stats command
et_irrig = ee.ImageCollection(et_irrig)
ir_tmp = ee.ImageCollection(ir_tmp)
ir_pr = ee.ImageCollection(ir_pr)
means_max_temp = ee.ImageCollection(mean_max)
et_march = ee.ImageCollection(mar_et)
tmp_march = ee.ImageCollection(mar_tmp)
prcp_march = ee.ImageCollection(mar_prcp)

In [ ]:
## ---------------------------------------------------------------------- ##
## 3. IMPORT THE DAYMET DATA FOR PRECIPITATION PRIOR TO IRRIGATION SEASON ##
## ---------------------------------------------------------------------- ##

# This will provide insight into antecedent moisture conditions for a POU.

years = np.arange(1987,2021)
ant_pr = []
for i in range(len(years)):
  daymet = ee.ImageCollection("NASA/ORNL/DAYMET_V4").filterDate((str(years[i]-1)+'-10-31'), start_end['StartDate'][i]) #get image collection
  prcp = daymet.select('prcp').map(lambda image: image.clip(subset)).sum().set({'system:index':str(i+1)}) #select the bands to analyze
  ant_pr.append(prcp) #calculate the mean across all pixels

ant_precip = ee.ImageCollection(ant_pr) #convert list of image to image collection for zonal stats command

precip_vis = {
  'min': 0,
  'max': 544,
  'palette': ['1621A2', 'white', 'cyan', 'green', 'yellow', 'orange', 'red'],
}

Map = emap.Map(center=(43.6150, -116.2023),zoom=8)
Map.addLayer(ant_precip, precip_vis, 'prcp')
Map

Map(center=[43.615, -116.2023], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchD…

In [ ]:
## ------------------------ ##
## 4. CALCULATE ZONAL STATS ##
## ------------------------ ##

# Allowed output formats: csv, shp, json, kml, kmz
# Allowed statistics type: MEAN, MAXIMUM, MINIMUM, MEDIAN, STD, MIN_MAX, VARIANCE, SUM

out_stats = os.path.join('/content/drive/MyDrive/Data/pod_pou_lulcc/data_input/extract_gridmet/climate_stats/JAtemp_stats.csv')
emap.zonal_statistics(means_max_temp, subset, out_stats, statistics_type='MEAN', scale=1000)

out_stats = os.path.join('/content/drive/MyDrive/Data/pod_pou_lulcc/data_input/extract_gridmet/climate_stats/ir_tmp_stats.csv')
emap.zonal_statistics(ir_tmp, subset, out_stats, statistics_type='MEAN', scale=1000)

out_stats = os.path.join('/content/drive/MyDrive/Data/pod_pou_lulcc/data_input/extract_gridmet/climate_stats/irrig_precip_stats.csv')
emap.zonal_statistics(ir_pr, subset, out_stats, statistics_type='MEAN', scale=1000)

out_stats = os.path.join('/content/drive/MyDrive/Data/pod_pou_lulcc/data_input/extract_gridmet/climate_stats/ant_precip_stats.csv')
emap.zonal_statistics(ant_precip, subset, out_stats, statistics_type='MEAN', scale=1000)

out_stats = os.path.join('/content/drive/MyDrive/Data/pod_pou_lulcc/data_input/extract_gridmet/climate_stats/et.csv')
emap.zonal_statistics(et_irrig, subset, out_stats, statistics_type='MEAN', scale=30)

out_stats = os.path.join('/content/drive/MyDrive/Data/pod_pou_lulcc/data_input/extract_gridmet/climate_stats/et_march.csv')
emap.zonal_statistics(et_march, subset, out_stats, statistics_type='MEAN', scale=30)

out_stats = os.path.join('/content/drive/MyDrive/Data/pod_pou_lulcc/data_input/extract_gridmet/climate_stats/tmp_march.csv')
emap.zonal_statistics(tmp_march, subset, out_stats, statistics_type='MEAN', scale=1000)

out_stats = os.path.join('/content/drive/MyDrive/Data/pod_pou_lulcc/data_input/extract_gridmet/climate_stats/prcp_march.csv')
emap.zonal_statistics(prcp_march, subset, out_stats, statistics_type='MEAN', scale=30)

Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/Data/Datasets from Bridget/pod_pou_lulcc/extract_gridmet/climate_stats/JAtemp_stats.csv
Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/Data/Datasets from Bridget/pod_pou_lulcc/extract_gridmet/climate_stats/ir_tmp_stats.csv
Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/Data/Datasets from Bridget/pod_pou_lulcc/extract_gridmet/climate_stats/irrig_precip_stats.csv
Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/Data/Datasets from Bridget/pod_pou_lulcc/extract_gridmet/climate_stats/ant_precip_stats.csv
Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/Data/Datasets from Bridget/pod_pou_lulcc/extract_gridmet/climate_stats/et.csv
Computing statistics ...
Generating URL ...
Please

In [ ]:
## ---------------------------------------------- ##
## 5. CREATE CLIMATE STAT FOR EACH POU AND EXPORT ##
## ---------------------------------------------- ##

years = np.arange(1987,2021)
ir_precip = pd.read_csv('/content/drive/MyDrive/Data/pod_pou_lulcc/data_input/extract_gridmet/climate_stats/irrig_precip_stats.csv')
ant_precip = pd.read_csv('/content/drive/MyDrive/Data/pod_pou_lulcc/data_input/extract_gridmet/climate_stats/ant_precip_stats.csv')
JA_temp = pd.read_csv('/content/drive/MyDrive/Data/pod_pou_lulcc/data_input/extract_gridmet/climate_stats/JAtemp_stats.csv')
irrig_temp = pd.read_csv('/content/drive/MyDrive/Data/pod_pou_lulcc/data_input/extract_gridmet/climate_stats/ir_tmp_stats.csv')
et_irrig = pd.read_csv('/content/drive/MyDrive/Data/pod_pou_lulcc/data_input/extract_gridmet/climate_stats/et.csv')
## MARCH INFO##
march_et = pd.read_csv('/content/drive/MyDrive/Data/pod_pou_lulcc/data_input/extract_gridmet/climate_stats/et_march.csv')
march_tmp = pd.read_csv('/content/drive/MyDrive/Data/pod_pou_lulcc/data_input/extract_gridmet/climate_stats/tmp_march.csv')
march_prcp = pd.read_csv('/content/drive/MyDrive/Data/pod_pou_lulcc/data_input/extract_gridmet/climate_stats/prcp_march.csv')


names = et_irrig['WaterRight']

for i in range(len(names)):
  df = pd.DataFrame(years, columns=['Year'])
  df['DIV_NAME'] = names[i]
  df['ant_prcp'] = ant_precip.iloc[i,0:34].values
  df['irrig_prcp'] = ir_precip.iloc[i,0:34].values
  df['irrig_temp'] = irrig_temp.iloc[i,0:34].values
  df['JuneAug_temp'] = JA_temp.iloc[i,0:34].values
  df['et'] = et_irrig.iloc[i,0:34].values
  df['Mar_et'] = march_et.iloc[i, 0:34].values
  df['Mar_tmp'] = march_tmp.iloc[i, 0:34].values
  df['Mar_prcp'] = march_prcp.iloc[i, 0:34].values
  out_path = os.path.join('/content/drive/MyDrive/Data/pod_pou_lulcc/data_output/extract_gridmet_out/final/'+names[i]+'_climate.csv')
  df.to_csv(out_path)